Details:

https://academic.oup.com/jnci/article/104/4/311/979947

> Single sample predictors (SSPs) and Subtype classification models (SCMs) are gene expression–based classifiers used to identify the four primary molecular subtypes of breast cancer (basal-like, HER2-enriched, luminal A, and luminal B). SSPs use hierarchical clustering, followed by nearest centroid classification, based on large sets of tumor-intrinsic genes. SCMs use a mixture of Gaussian distributions based on sets of genes with expression specifically correlated with three key breast cancer genes (estrogen receptor [ER], HER2, and aurora kinase A [AURKA]). The aim of this study was to compare the robustness, classification concordance, and prognostic value of these classifiers with those of a simplified three-gene SCM in a large compendium of microarray datasets.

AURKA

ER is ESR1 (Source: https://www.genecards.org/cgi-bin/carddisp.pl?gene=ESR1)

HER2 is ERBB2 (Source: https://www.genecards.org/cgi-bin/carddisp.pl?gene=ERBB2)

In [156]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Customize this for each notebook

In [157]:
OUTPUT_DIR='Three-Gene-Model-Lab-Demo/'
OPTIONS={'seed':3}
PREFIX="_".join([f"{key}={OPTIONS[key]}" for key in OPTIONS.keys()])
RESULTS={}
PREFIX

'seed=3'

In [158]:
from pathlib import Path
home = str(Path.home())

In [159]:
KNOWLEDGE_LIB=f'{home}/knowledgelib'

In [160]:
from IPython.display import display, Markdown, Latex
import sys
sys.path.insert(0,f'{KNOWLEDGE_LIB}')
import pyknowledge
import pandas as pd
import scipy.io
import pandas as pd
import numpy as np
import joblib

## Load the input data

In [161]:
## Customize this load to read in the data and format it with the correct columns
def load_data_all():
    mat = scipy.io.loadmat("/disk/metabric/BRCA1View20000.mat")
    #gene_labels = open("/disk/metabric/gene_labels.txt").read().split("\n")
    gene_labels = [g[0] for g in mat['gene'][0]]
    df = pd.DataFrame(mat['data'].transpose(), columns=gene_labels)
    [n_dim, n_sample] = df.shape
    for i in range(n_dim):
        m1 = min(df.iloc[:,i])
        m2 = max(df.iloc[:,i])
        df.iloc[:,i] =(df.iloc[:,i] - m1)/(m2 - m1)
    df['target'] = mat['targets']
    df['Subtype'] = df.target.map({1:'Basal',2:'HER2+',3:'LumA',4:'LumB',5:'Normal Like',6:'Normal'})
    df['color'] = df.target.map({1:'red',2:'green',3:'purple',4:'cyan',5:'blue',6:'green'})
    df['graph_color'] = df.target.map({1:'#FFFFFF',2:'#F5F5F5',3:'#FFFAFA',4:'#FFFFF0',5:'#FFFAF0',6:'#F5FFFA'})
    df = df.set_index(np.arange(len(df)))
    
    return df

In [162]:
df_all = load_data_all()

In [163]:
df_all.Subtype.value_counts() # basal-like, HER2-enriched, luminal A, and luminal B

LumA           721
LumB           491
Basal          330
HER2+          239
Normal Like    202
Normal         150
Name: Subtype, dtype: int64

## Knowledge

#### Genes

In [164]:
knowledge_genes = ["ERBB2","ESR1","AURKA"]

In [165]:
genes_df_all = df_all[knowledge_genes+["Subtype"]]
genes_df_all.head()

,ERBB2,ESR1,AURKA,Subtype
0,6.277414,0.247511,0.022905,Normal
1,6.767912,0.748999,0.299310,LumB
2,7.096390,0.828680,0.209436,LumB
3,7.185442,0.688980,0.396504,Normal Like
4,7.179097,0.803549,0.371463,LumA


In [166]:
means = genes_df_all.groupby('Subtype').mean()
medians = genes_df_all.groupby('Subtype').median()
stdevs = genes_df_all.groupby('Subtype').std()

means

,ERBB2,ESR1,AURKA
Subtype,,,
Basal,6.953369,0.150253,0.460046
HER2+,7.888029,0.314071,0.414654
LumA,7.174146,0.691527,0.242708
LumB,7.154050,0.715256,0.393492
Normal,6.820962,0.448411,0.090667
Normal Like,7.120211,0.496876,0.221378


In [167]:
medians

,ERBB2,ESR1,AURKA
Subtype,,,
Basal,6.856068,0.104206,0.470450
HER2+,8.044117,0.222098,0.404954
LumA,7.147807,0.706710,0.231855
LumB,7.095976,0.735066,0.384867
Normal,6.907045,0.430171,0.072994
Normal Like,7.065202,0.519050,0.210372


In [168]:
stdevs

,ERBB2,ESR1,AURKA
Subtype,,,
Basal,0.553819,0.132523,0.122079
HER2+,0.638267,0.238140,0.105670
LumA,0.291271,0.129719,0.094623
LumB,0.452018,0.122941,0.108063
Normal,0.359079,0.154813,0.078081
Normal Like,0.446458,0.188961,0.102247


In [169]:
baseline = 'Basal'
baseline_means = means.loc[baseline]
baseline_medians = medians.loc[baseline]
baseline_stdevs = stdevs.loc[baseline]
baseline_means

ERBB2    6.953369
ESR1     0.150253
AURKA    0.460046
Name: Basal, dtype: float64

## Scale

In [170]:
df_subtype = genes_df_all.loc[genes_df_all['Subtype'] != baseline]
df = df_subtype.drop('Subtype',axis=1).subtract(baseline_means,axis=1).divide(baseline_stdevs,axis=1)
df = df.join(df_subtype[['Subtype']])
df

,ERBB2,ESR1,AURKA,Subtype
0,-1.220536,0.733895,-3.580797,Normal
1,-0.334869,4.518054,-1.316652,LumB
2,0.258245,5.119315,-2.052850,LumB
3,0.419041,4.065156,-0.520499,Normal Like
4,0.407585,4.929682,-0.725618,LumA
...,...,...,...,...
2127,0.287164,4.210002,-1.201067,Normal Like
2128,0.086196,4.419438,-2.572388,LumA
2129,1.267920,1.580464,-1.344704,Normal Like
2130,0.765957,1.907787,-1.393883,HER2+


## EDA

### Each gene individually

In [171]:
source = df.join(df_all[['target']]).melt(id_vars=['Subtype','target'])
source.columns = ["Subtype","target","Gene","Value"]
counts = source.groupby('Subtype')['target'].count().to_frame()
counts.columns = ['Count']
source = source.set_index('Subtype').join(counts).reset_index()
import altair as alt
alt.data_transformers.disable_max_rows()

g = alt.Chart(source).transform_calculate(
    pct='1 / datum.Count'
).mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('Value:Q', bin=alt.Bin(maxbins=100)),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%'),stack=None),
    alt.Color('Subtype:N'),
    row='Gene:N'
)
g

alt.Chart(...)

In [172]:
directions = df.drop('Subtype',axis=1)>0
pattern_counts = directions.value_counts()
pattern_counts

ERBB2  ESR1   AURKA
True   True   False    1105
False  True   False     403
True   True   True      144
       False  False      74
False  True   True       44
True   False  True       25
False  False  False       7
              True        1
dtype: int64

In [173]:
pattern_subtype_counts = directions.join(df[['Subtype']]).groupby(knowledge_genes)['Subtype'].value_counts()
pattern_subtype_frac = pattern_subtype_counts.divide(pattern_counts)
pattern_df = pattern_subtype_frac.to_frame().join(pattern_subtype_counts)
pattern_df.columns=['Fraction','Count']
pattern_df.sort_values(by='Fraction',ascending=False)

Fraction  Count
ERBB2 ESR1  AURKA Subtype                     
False False True  HER2+        1.000000      1
True  False True  HER2+        1.000000     25
False True  True  LumB         0.840909     37
True  False False HER2+        0.797297     59
      True  True  LumB         0.597222     86
False False False HER2+        0.571429      4
True  True  False LumA         0.523077    578
False False False Normal Like  0.428571      3
      True  False LumA         0.307692    124
True  True  True  HER2+        0.298611     43
False True  False LumB         0.277916    112
True  True  False LumB         0.231674    256
False True  False Normal       0.220844     89
                  Normal Like  0.168734     68
True  False False Normal Like  0.162162     12
      True  False Normal Like  0.105882    117
            True  LumA         0.097222     14
            False HER2+        0.085068     94
False True  True  HER2+        0.068182      3
True  True  False Normal       0.054299     60
False True  True  LumA         0.045455      2
True  False False LumA         0.040541      3
False True  False HER2+        0.024814     10
            True  Normal Like  0.022727      1
                  Normal       0.022727      1
True  True  True  Normal Like  0.006944      1

In [174]:
patterns = set([
    (False,True,True),
    (True,False,False)
 #   (True,True,False) # Maybe?
])

In [175]:
mask = []
for ix1 in df.index:
    directions1 = tuple(directions.loc[ix1])
    if directions1 not in patterns:
        mask.append(False)
    else:
        mask.append(True)

In [176]:
from IPython.display import Image

import networkx as nx

#G = nx.Graph()
#for ix in df_all.index:
#    c = 'white'
#    G.add_node(ix,color='black',style='filled',fillcolor=c)

df_mask = df.loc[mask]
directions_mask = directions.loc[mask]

In [177]:
A = pd.DataFrame(index=df_all.index,columns=df_all.index)
columns = directions_mask.columns
pts = directions_mask.reset_index().set_index(list(columns))
patterns_list = list(patterns)
for pattern in patterns_list:
    ixs = pts.loc[pattern,'index']
    A.loc[ixs,ixs] = 1
np.fill_diagonal(A.values, np.NaN)
A.stack()

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


17    43      1
      69      1
      101     1
      253     1
      280     1
             ..
2117  2028    1
      2041    1
      2051    1
      2077    1
      2115    1
Length: 7294, dtype: object

In [178]:
#A = A.loc[A.fillna(0).sum()!=0]
#A = A.loc[:,A.index]

In [179]:
A.shape

(2133, 2133)

In [180]:
A.to_csv(f'{OUTPUT_DIR}/A.csv')

**You will only need to proceed after this point if you want to visualize the graphs**

**Proceed with caution as they might be large**

In [127]:
G = nx.from_pandas_adjacency(A.fillna(0), create_using=nx.Graph)

In [128]:
def save(A,file="graph.png"):
    g = A.draw(format=file.split(".")[-1], prog='dot')
    open(file,"wb").write(g)
    return Image(g)

#pos = nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')
#A = nx.nx_agraph.to_agraph(G)
#A.graph_attr["rankdir"] = "LR"
# draw it in the notebook
#save(A,file=f"{OUTPUT_DIR}{PREFIX}_graph.png")

In [129]:
!mkdir {OUTPUT_DIR}{PREFIX}_graphs

mkdir: cannot create directory ‘Three-Gene-Model-Lab-Demo/seed=3_graphs’: File exists


In [130]:
graphs = list(G.subgraph(c).copy() for c in nx.connected_components(G))

for i,graph in enumerate(graphs):
    nodes = list(graph.nodes())
    if len(nodes) > 1:
        pos = nx.drawing.nx_agraph.graphviz_layout(graph, prog='dot')
        AG = nx.nx_agraph.to_agraph(graph)
        AG.graph_attr["rankdir"] = "LR"
        # draw it in the notebook
        save(AG,file=f"{OUTPUT_DIR}{PREFIX}_graphs/graph_{i}.png")

KeyboardInterrupt: 